# Chained Functions Using OpenAPI Specification

The majority of the tools we use over the internet are powered by RESTful APIs. Giving GPT the power to call them opens up a world of possibilities. This notebook demonstrates how GPTs can be used to intelligently call APIs. It leverages OpenAPI specifications and chained function calls. 

The OpenAPI Specification (OAS) is a universally accepted standard for describing the details of RESTful APIs in a format that machines can read and interpret. It enables both humans and computers to understand the capabilities of a service without direct access to the source code.

This notebook is divided into two main sections: 

1. The first section involves converting a sample OpenAPI spec into a list of function definitions along with their expected arguments. This is done by parsing the OpenAPI spec and extracting the necessary information. 
2. The second section involves taking the list of functions generated in the first step, along with a user instruction, and executing the functions sequentially. This is done by feeding the function definitions and the user instruction to the Chat completion API, which generates the JSON objects for function calls.

In [ ]:
!pip install -q jsonref
!pip install -q openai

In [2]:
import os
import json
import jsonref
import openai
import requests
from pprint import pp

openai.api_key = os.environ["OPENAI_API_KEY"]

## Converting OpenAPI Specifications into OpenAI Functions

The example OpenAPI spec we use here was created using `gpt-4`. Here we will transform the spec into a set of function definitions that can be supplied to the chat completion API. The model, based on the provided user instructions, generates a JSON object containing the necessary arguments to call these functions.

Here's a brief overview of the dialogue that led to its creation:

```
User: Let's generate a fake Swagger openai.json for a "froge" character database.
Assistant: Provides a draft Swagger specification.
User: Use OpenAPI spec and also add endpoints for getting and updating the name for a froge and getting froge by name.
Assistant: Updates the specification to OpenAPI 3.0.0, adds endpoints for updating a Froge's name, and getting a Froge by name.
User: Could you also add a function name as operationId to it?
Assistant: Adds operationId for function names to each endpoint.
```

Before we proceed, let's inspect this generated spec. OpenAPI specs include details about the API's endpoints, the operations they support, the parameters they accept, the requests they can handle, and the responses they return. The spec is defined in JSON format.

The endpoints in the spec include operations for:

- listing all "froge" characters
- creating a new "froge" character
- retrieving a "froge" character by ID
- deleting a "froge" character by ID
- updating a "froge" character's name by ID
- retrieving a "froge" character by name.

Each operation in the spec has an operationId, a unique string used to identify an operation, which we will use as the function name when we parse the spec into function specifications. The spec also includes schemas that define the data types and structures of the parameters for each operation. These schemas will be used to validate the input parameters when we call the functions.

You can see the schema here:


In [3]:
url = "https://gist.githubusercontent.com/shyamal-anadkat/d44674a87778796222bdb8fa9158ad47/raw/030d173d53c55d806a93976705cf1c5f9e9c5240/frogeapi.json"
openapi_spec = jsonref.loads(requests.get(url).content)

display(openapi_spec)

{'openapi': '3.0.0',
 'info': {'version': '1.0.0',
  'title': 'Froge Character API',
  'description': 'An API for managing froge character data'},
 'paths': {'/froge': {'get': {'summary': 'List all froge characters',
    'operationId': 'listFroges',
    'responses': {'200': {'description': 'A list of froge characters',
      'content': {'application/json': {'schema': {'type': 'array',
         'items': {'type': 'object',
          'properties': {'id': {'type': 'string'},
           'name': {'type': 'string'},
           'age': {'type': 'integer'}},
          'required': ['name', 'age']}}}}}}},
   'post': {'summary': 'Create a new froge character',
    'operationId': 'createFroge',
    'requestBody': {'required': True,
     'content': {'application/json': {'schema': {'type': 'object',
        'properties': {'id': {'type': 'string'},
         'name': {'type': 'string'},
         'age': {'type': 'integer'}},
        'required': ['name', 'age']}}}},
    'responses': {'201': {'description':

Now that we have a good understanding of the OpenAPI spec, we can proceed to parse it into function specifications.

The main objective of `parse_functions` is to generate a list of functions, where each function is represented as a dictionary containing the following keys:
- 'name': This corresponds to the operation identifier of the API endpoint as defined in the OpenAPI specification.
- 'description': This is a brief description or summary of the function, providing an overview of what the function does.
- 'parameters': This is a schema that defines the expected input parameters for the function. It provides information about the type of each parameter, whether it is required or optional, and other related details.

The output of this function is a list of such dictionaries, each representing a function defined in the OpenAPI specification.

In [4]:
def parse_functions(openapi_spec):    

    # Initialize an empty list to store the function specifications
    functions = []

    # Iterate through the paths and methods specified in the OpenAPI spec
    for path, methods in openapi_spec["paths"].items():
        for method, spec_with_ref in methods.items():
            # Replace any JSON references in the spec
            spec = jsonref.replace_refs(spec_with_ref)

            # Extract the operationId which will be used as the function name
            function_name = spec.get("operationId")
            
            # Gather the description, request body and parameters from the spec
            desc = spec.get("description") or spec.get("summary", "")
            req_body = spec.get("requestBody", {}).get("content", {}).get("application/json", {}).get("schema")
            params = spec.get("parameters", [])
            
            # Initialize an empty schema
            schema = {"type": "object", "properties": {}}
            
            # If a request body is defined, add it to the schema
            if req_body:
                schema["properties"]["requestBody"] = req_body
            
            # If parameters are defined, add them to the schema
            if params:
                param_properties = {param["name"]: param["schema"] for param in params if "schema" in param}
                schema["properties"]["parameters"] = {"type": "object", "properties": param_properties}
            
            # Append the function specification to the list of functions
            functions.append({"name": function_name, "description": desc, "parameters": schema})

    # Return the list of function specifications
    return functions

# Parse the OpenAPI spec to get the function specifications
functions = parse_functions(openapi_spec)


for function in functions:
    pp(function)
    print()


{'name': 'listFroges',
 'description': 'List all froge characters',
 'parameters': {'type': 'object', 'properties': {}}}

{'name': 'createFroge',
 'description': 'Create a new froge character',
 'parameters': {'type': 'object',
                'properties': {'requestBody': {'type': 'object',
                                               'properties': {'id': {'type': 'string'},
                                                              'name': {'type': 'string'},
                                                              'age': {'type': 'integer'}},
                                               'required': ['name', 'age']}}}}

{'name': 'getFrogeById',
 'description': 'Retrieve a froge character by ID',
 'parameters': {'type': 'object',
                'properties': {'parameters': {'type': 'object',
                                              'properties': {'id': {'type': 'string'}}}}}}

{'name': 'deleteFroge',
 'description': 'Delete a froge character by ID',
 'parameters': {'

## Orchestrating Sequential Function Calls


Now that we have these function definitions that we have derived from the OpenAPI spec, we can leverage them to orchestrate a series of calls to the `gpt-3.5-turbo-16k-0613` model. 

The model will determine the sequence of functions to call based on the user's input and the available function specifications. 

It's important to note that the chat completions API does not execute the function; instead, it generates the JSON that you can use to call the function in your own code.

In [5]:
SYSTEM_PROMPT = """
You are a helpful assistant. 
Respond to the following prompt by using function_call and then summarize actions. 
Ask for clarification if a user request is ambiguous.
"""

USER_PROMPT = """
Instruction: Get all the froges. 
Then create a new 2-year old froge named dalle3, with a random numerical id. 
Then delete froge with id 2456.
"""

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT}
]

# Maximum number of chained calls allowed to prevent infinite or lengthy loops
MAX_CHAINED_CALLS = 5

def get_openai_response(functions, messages):
    return openai.ChatCompletion.create(
        model='gpt-3.5-turbo-16k-0613',
        functions=functions,
        function_call="auto", # "auto" means the model can pick between generating a message or calling a function.
        temperature=0,
        messages=messages
    )

def process_chained_calls(functions, messages):
    stack = 0
    while stack < MAX_CHAINED_CALLS:
        response = get_openai_response(functions, messages)
        message = response["choices"][0]["message"]
        
        if message.get("function_call"):
            print(f"\nFunction call #: {stack + 1}")
            pp(message["function_call"])
            messages.append(message)
            stack += 1
        else:
            print("\nMessage:")
            print(message["content"])
            break
    
    if stack >= MAX_CHAINED_CALLS:
        print(f"Reached max chained function calls: {MAX_CHAINED_CALLS}")

process_chained_calls(functions, messages)


Function call #: 1
{'name': 'listFroges',
 'arguments': '{}'}

Function call #: 2
{'name': 'createFroge',
 'arguments': '{\n'
              '  "requestBody": {\n'
              '    "id": "1234",\n'
              '    "name": "dalle3",\n'
              '    "age": 2\n'
              '  }\n'
              '}'}

Function call #: 3
{'name': 'deleteFroge',
 'arguments': '{\n  "parameters": {\n    "id": "2456"\n  }\n}'}

Message:
Actions:
1. Retrieved all the froges.
2. Created a new froge named "dalle3" with an age of 2 and a random numerical id.
3. Deleted the froge with id 2456.



### Conclusion

We have demonstrated how to convert OpenAPI specs into function specifications that can be given to GPT for it to intelligently call them, and shown how these can be chained together to perform complex operations.

Possible extensions of this system could include handling more complex user instructions that require conditional logic or looping, integrating with real APIs to perform actual operations, and improving error handling and validation to ensure the instructions are feasible and the function calls are successful.